In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pickle
import pandas as pd
pd.set_option('max_colwidth',200)
raw_data = pd.read_pickle("/content/drive/MyDrive/project_rap/raw_data_df.pkl")

In [3]:
raw_data.head()

,lyrics
Eminem,"[Announcer: Jeff Bass & Eminem]\nThis is another public service announcement\nBrought to you, in part, by Slim Shady\nTell 'em I don't give a fuck\nSlim Shady does not give a fuck what you think\n..."
Jay Z,"[Produced by The Hitmen & Knobody]\n[Skit: Pain In Da Ass]\nAh shit, okay, okay, alright\nBig man! You wanna make some big bucks, huh?\nLet's see how tough you are\nYou know something about cocain..."
Kanye West,[Produced by Kanye West]\n\n[Intro: Kanye West]\nUhh.. uhh\nUhh.. uhh\n\n[Chorus: Kanye West]\nGood morning!\nGood morning!\nGood morning!\nGood morning!\n\n[Verse 1: Kanye West]\nWake up Mr. West...
Kendrick Lamar,"[Intro]\nLord God, I come to you a sinner\nAnd I humbly repent for my sins\nI believe that Jesus is Lord\nI believe that You raised him from the dead\nI would ask that Jesus come to my life\nAnd b..."
Tupac,"[Male Reporter]\nAt 12:25 AM Wednesday, 2Pac was on his way into a time square building to record at an eight-floor studio with another rapper. But in the lobby, Shakur was shot several times, inc..."


In [4]:
# function to cleare dataframe
import re
import string

def clean_text_round1(text):
    '''Remove text in square brackets, remove commas, remove numbers and make text lowercase.'''
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

round1 = lambda x: clean_text_round1(x)

In [5]:
# Let's take a look at the updated text
data_clean = pd.DataFrame(raw_data.lyrics.apply(round1))
data_clean.head()

,lyrics
Eminem,\nthis is another public service announcement\nbrought to you in part by slim shady\ntell em i dont give a fuck\nslim shady does not give a fuck what you think\ntell em to suck it\nif you dont lik...
Jay Z,\n\nah shit okay okay alright\nbig man you wanna make some big bucks huh\nlets see how tough you are\nyou know something about cocaine digame\nyou kidding me or what man\ntheres a bunch of colombi...
Kanye West,\n\n\nuhh uhh\nuhh uhh\n\n\ngood morning\ngood morning\ngood morning\ngood morning\n\n\nwake up mr west mr west\nmr fresh mister by himself hes so impressed\ni mean damn did you even see the test\...
Kendrick Lamar,\nlord god i come to you a sinner\nand i humbly repent for my sins\ni believe that jesus is lord\ni believe that you raised him from the dead\ni would ask that jesus come to my life\nand be my lor...
Tupac,\nat am wednesday was on his way into a time square building to record at an eightfloor studio with another rapper but in the lobby shakur was shot several times including two graze wounds to th...


In [6]:
# Apply a second round of cleaning
def clean_text_round2(text):
    '''Get rid of some additional punctuation and non-sensical text that was missed the first time around.'''
    text = re.sub('[‘’“”…]', '', text)
    text = re.sub('\n', ' ', text)
    return text

round2 = lambda x: clean_text_round2(x)

In [7]:
# Let's take a look at the updated text
data_clean = pd.DataFrame(data_clean.lyrics.apply(round2))
data_clean.head()

,lyrics
Eminem,this is another public service announcement brought to you in part by slim shady tell em i dont give a fuck slim shady does not give a fuck what you think tell em to suck it if you dont like it y...
Jay Z,ah shit okay okay alright big man you wanna make some big bucks huh lets see how tough you are you know something about cocaine digame you kidding me or what man theres a bunch of colombians com...
Kanye West,uhh uhh uhh uhh good morning good morning good morning good morning wake up mr west mr west mr fresh mister by himself hes so impressed i mean damn did you even see the test you got ds muth...
Kendrick Lamar,lord god i come to you a sinner and i humbly repent for my sins i believe that jesus is lord i believe that you raised him from the dead i would ask that jesus come to my life and be my lord and ...
Tupac,at am wednesday was on his way into a time square building to record at an eightfloor studio with another rapper but in the lobby shakur was shot several times including two graze wounds to the...


In [8]:
data_clean.to_pickle("data_clean_df.pkl")

<h4>Lemmetization</4>

In [9]:
# import Lemmatizer
import nltk
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer 

# Init the Wordnet Lemmatizer
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [10]:
# Lemmatize with POS Tag
from nltk.corpus import wordnet

def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

In [11]:
# my final list of rappers whose album will be analysed
rappers = ['Eminem','Jay Z','Kanye West', 'Kendrick Lamar','Tupac']

# see the dataset size before applying lemmatizer
for r in rappers:
    l1 =len(data_clean.lyrics[r])
    print(r + " len = " + str(l1))

Eminem len = 43294
Jay Z len = 32921
Kanye West len = 21719
Kendrick Lamar len = 44634
Tupac len = 33435


In [12]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [13]:
# 1. Init Lemmatizer
lemmatizer = WordNetLemmatizer()

# 2. Lemmatize dataset
for rapper, text in data_clean.iterrows():
    sentence = text[0]
    lemmatized_output = ' '.join([lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in nltk.word_tokenize(sentence)])
    data_clean.lyrics[rapper] = lemmatized_output
    lemmatized_output = ''

In [14]:
# see the dataset size after applying lemmatizer
for r in rappers:
    l1 =len(data_clean.lyrics[r])
    print(r + " len = " + str(l1))

Eminem len = 42306
Jay Z len = 31836
Kanye West len = 21100
Kendrick Lamar len = 43441
Tupac len = 32309


<h4>Organizing Data</h4>

In [15]:
trash = 'im cause like aap ac acabei acabou ace eu acerto ache еu agh aha ahh ak al alguém alameda alaskan alegre algo ali altj amaze andre anduh annoy ano ante anus aos apenas ayayayah ay ayo bono boo boob booyaow bêbadas cabeça bêbada caixão caia cara caras cbs cd ce coração cuzão cyhi césar código  acima actionsll água eu zigzag zula youuvvampire youknowwho	youknowwhat yo yessir diz dizem dizendo dizer dj doe doodoodoodoodoot doodoodoodoot dou dyke eminem eminemll então estão faça fi flórida fml fo friendzwatchutola frágil ft fuckin gegegeyeahhh geez gelados graça  yer yen yeezy	yelawolf ye yatold	yall	yak yknow yi  know aalex dont aahah aaow ah ahah ahahah ahahahahahahahaha ahahahh	ahhahahah	ahhahaowww	amo ass aye ayy ayyuh bette cha chadachashashashashasha citys cocacola comin cyar cómo dijo eh eheh fikki gahhlee gimme givin gjoob ha haffi haoh haooh harlow hayworth hee heehee heeheehee heh hideandseek hmm ho hoo hov huh iana iiiiiiiiihiii ima imma inez juju kickin lalalalalalala la ma mckenzie meh mm mmm mmmm mmmmm mmmmmm mmmmmmmmmmmmmmm mojo molly naa nanana nananana nicest nonono nononononono nuh oath nononononono ohno ohoh ohohoh ol ono ooh oohooh oooh ooooooohhh ahoo just shit thats aint make fuck say na time man nigga right yeah life yourselves yous yukmouth yummy yumyum yyeah papapapapapapapa papapapapa owwwwww ow ouw shoochachoocha shoochachoo wah  yyyou zach zachs zakia zee zhané zion haffa hah haha hahaha hahahaha heeee hehe hennessy hennessey hennesey heyyayyaahhyy heyyyy hm há ia inn inning jiggyjigga jv já le ld ligação mami mamis meeearl mhm mo motherfucker motherfuckin motto mr mtv moët muhfuckers mês música nana narcissus negócio nem nnow norman nova novo nsx nsync nun não og oh oj ok okay olá ooooh oops opa otis pa pac pacin pantyhose pas passado pastellé passou peru pharaoh playa playboi pussy página páscoa quê re rza sabe saber sadnesskkanye schizophrenic schwarzenegger ser seu seus seventeenyearold shakur shoobydedoowop showbiz sia skibbedybebop sono sorrow sue swingerjjames sylvan são só sóbria também tava ththat tyty tá tão tô uchis ufo ugh uh uhh uhhuh uhhuhhum uhuh um uma ummm uzi vai vcryzembedshare vd veggie vejo vestido whoa whod whoo woo wooee wzrd xannies xx xxl eu zeppelin zsa aa aaah aaahhh aaliyah aames aanything ça écoute époque étaient était bneler nanamıyorum scratch şte beni hatırladın mı yedi infaz beni hatırladın mı hiç bir pişmanlığım yok beni hatırladın mı ben fazla güçlüyüm beni hatırladın mı bombaları atıyorum hiroşimaya atılanlar gibi   bu onun için x seni delirtirim çünkü ben mikrofonu kapıp ortadan kaybolabilirim down sendromu gibi gizlen ve insanların içinde dolaş sınırları olmayan biriyim hanginiz beni tanımlayabilir evrenin sağlığı için hiç bir prova olmadan bu tartışmalı kelimeleri sızdırıyorum yarışmak isteyeceğin tiplerden değilim çünkü çünkü senin kıçına çarpıyorum hani şu reklamdaki sürtüğe çarpan tren gibi bu reklam tvde yasaklandı ağır keskin nişancıyım seni vuruyorum başının ortadan ikiye ayrıldığını seyredebilirsin lokomotif gibi geliyoruz çılgın bir şekildeün devamı i̇şte içi boş  kalibrelik kurşunlar   beni hatırladın mı silahlarınızı havaya atın beni hatırladın mı çarp çarp beni hatırladın mı zenci geri çekil amına koyim beni hatırladın mı chkachkaonyx  zenciler sorularına hayır cevabını aldılar evet hayır dedim ama bu sanki hayırhayırhayır gibiydi hayat bir sürtük tanıştım onunla eğer zincirleri onun eline verirsen siktir git karşıma çıkmak istiyorsan daha iyinin daha iyisinin de daha iyisi olmalısın i̇şte ben ki usta rapçi akıttığı kanlara bakıyor sen ölünün de ölüsüsün artık sana para değil doktor gelecek sayemde kavgalı olduklarıma sırayla silahımla beraber gidiyorum kanıt mı hayır asla ardımda iz bırakmam ne kadar rapçi varsa hepsinin ruhuna az çok sahibim sev ya da nefret et ama durum bu annem müzik piyasası tarafından tecavüze uğradı ve ben doğdum ben en manyak zenciyim sana söylemiştim şu lezbiyen kaltaklardan oluşan total grubundan bile fazla kaltağa sahibim benle dalaşmak mı istiyorsun pbbt ölüm kontratını imzalamış olursun benim adım bu zencinin bunu söylediğine i̇nanamıyorum olmalıydı büyük ihtimalle katil olmadığımı söyleyecekler ama ben kendimi öldürüyorum ölüm içiyorum mutfak tezgahında topluluk ortasında kaltakları kondomsuz beceriyorum yani sana yaptığım şeyleri zerre umursamam çevremde bir tutam ot karşılığı tutabileceğim çok kiralık katil var ne yapmak istersin i̇bneler biz glock taşıyan adamlarız polis bizi kelepçeleyene kadar gürültü başlatırız ve uçak bombası bırakırız zaman doldu fahişeler bize dokunamazsınız beni öldürmek isteyenlerin kafamı isteyenlerin olduğunu duyuyorum bunların benim söyleyebileceğim şeyler olduğunu mu sanıyorsun   beni hatırladın mı sikimde değil beni hatırladın mı evet seni de sikeyim beni hatırladın mı kısa ve kurnazım beni hatırladın mı ben shadyim  ne zaman dışarı çıksam ateş etmeye devam ediyorum vatan için savaşmak anlamında değil kulübe her gittiğim anı kastettim gerizekalı sikik imajımı düzeltmeye çalışıyorum bu yüzden de sikik eleştirmenlere söz veriyorum ki altı dakika boyunca sikik demeyeceğim  dakika slim shady başla bebeğimin annesi beni kızgın bir sarışın yaptı bu sinir bana bir şarkı bile yaptırdı şarkıda onu öldürüyordum ve şarkıya hailieyi de koydum belki de hatalıydım hep böyle delice soruları düşünüp duruyorum beynimin içinde ama ben çılgın bi anneyle de sıkışıp kalmıştım gerçekten annen dediğin kadar hap kullanıyor mu eve geldim arkadaki camları birileri kırmış olmalı ve iki tane dolu makineli tüfeğimi ve de her iki yağmurluğumu çalmışlardı kötü kötü rüyalar görüyorum piknik sahnelerinden i̇ki velet var on altı yaşındalar  silahları ve onar tane dolu şarjörleri var ve altışar çocuk bunların yüzünden ölüyor bill clintonın sokakları güya temizleyeceğini söylediği konuşmalarında slim de suçlandı sikerim böyle işi pbbt – siz ibneler volkanda yanıp kül olabilirsiniz ve de cehennemde bir şişe benzin ve çakmakla yeniden ortaya çıkabilirsiniz aftermath dre silahı kap nerede olduğunu göster neye bakmıştın zenci   beni hatırlayamadın mı beni hatırladın mı beni hatırladın mı beni hatırladın mı guegueguess whos back hi mom \u200b\u200bscratch\u200b\u200b guess whos back \u200b\u200bscratch\u200b\u200b gue— \u200b\u200bscratch\u200b\u200b guess whos back  \u200b\u200bscratch\u200b\u200b guess whos back gue gueguegue guess whos back dr dre \u200b\u200bscratch\u200b\u200b guess whos back back back \u200b\u200bscratch\u200b\u200b back \u200b\u200bscratch\u200b\u200b'
trash_words = trash.split()

In [16]:
# create document-term matrix for data_clean(lemmetized) dataframe
# also with stop words
from sklearn.feature_extraction import text 
from sklearn.feature_extraction.text import CountVectorizer

# Add new stop words
stop_words = text.ENGLISH_STOP_WORDS.union(trash_words)

# Recreate document-term matrix
cv = CountVectorizer(stop_words=trash_words)
data_cv = cv.fit_transform(data_clean.lyrics)
clean_data_dtm = pd.DataFrame(data_cv.toarray(), columns=cv.get_feature_names())
clean_data_dtm.index = data_clean.index

# pickle collected dataset for later use
clean_data_dtm.to_pickle("clean_data_df.pkl")

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [17]:
ls

clean_data_df.pkl  data_clean_df.pkl  drive/  sample_data/


In [18]:
clean_data_dtm

,abandon,able,about,above,abuse,abusin,accent,accountable,accountant,accumulates,accustom,acid,acquire,acredita,across,act,actin,action,active,activist,actuality,adam,adapt,add,addict,address,adina,adivinhar,adjust,administer,admire,admit,adolescent,adora,adrenaline,advance,advantage,adversary,advice,aeroplane,...,world,worldft,worm,worried,worry,worth,would,woulda,wouldnt,wound,wrap,wrestle,wrestler,wrinkle,wrist,wristwatch,write,writeoff,wrong,wronger,year,yell,yellin,yellow,yes,yesterday,yet,york,you,youd,youll,young,youngin,your,youre,yours,yourself,youth,youve,zone
Eminem,0,1,24,0,1,0,1,0,0,0,0,2,0,0,1,3,0,0,0,0,0,0,0,2,0,2,0,0,0,0,1,1,0,0,1,0,0,0,0,0,...,7,0,0,0,0,1,18,8,2,0,0,1,1,0,7,1,7,0,1,0,5,0,0,0,9,0,2,0,245,1,6,1,0,60,13,3,1,0,1,0
Jay Z,0,0,5,0,0,0,0,1,0,1,1,0,1,0,0,2,0,0,0,0,0,0,1,0,1,0,0,0,0,1,1,0,0,0,0,2,1,0,0,0,...,2,0,1,0,1,0,2,0,0,0,0,0,0,1,1,0,1,1,1,0,4,0,0,0,10,0,2,2,216,0,3,0,0,65,11,1,0,1,2,1
Kanye West,0,0,1,0,0,0,0,0,0,0,0,0,0,2,0,11,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,...,1,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,1,4,0,0,0,0,2,0,1,0,107,0,1,1,0,14,7,1,0,0,0,0
Kendrick Lamar,0,0,35,0,0,1,1,0,0,0,1,1,1,0,0,1,1,2,1,1,1,0,0,0,1,0,1,0,1,0,0,1,1,0,0,0,0,0,1,1,...,9,1,0,0,0,1,10,0,0,0,1,0,0,0,0,0,7,0,4,0,3,3,1,1,0,1,2,0,240,0,12,7,1,72,9,4,0,0,1,0
Tupac,1,0,7,3,0,0,0,0,1,0,0,0,0,0,0,4,0,2,0,0,0,0,0,0,2,1,0,0,0,0,0,0,0,0,0,0,0,2,0,0,...,34,0,0,1,4,0,2,0,2,1,0,0,0,0,0,0,0,0,7,0,10,1,0,0,2,1,0,1,117,0,0,28,0,30,8,1,1,1,2,0


In [19]:
clean_data_dtm.to_pickle("clean_data_dtm.pkl")

In [20]:
a = clean_data_dtm
type(a)

pandas.core.frame.DataFrame

In [21]:
import pandas as pd
a.to_excel('export_dataframe.xlsx')
